## Description

We Achieved 0.883664 with this model at 69000 iterations (z value not number of epochs which is 47) 

Net Prameters:
1. Batch size = 16 with shuffling but gradient update every four batches thus makig it viturally a btach size of 64

2. Learning Rate = Dynamic using Adam

3. Input dimensions = [-1,3,240,320]

4. Output dimensions = [-1,4]          
    ie.[batch_size,x1,y1,x2,y2]

Image transformations:
1. Resizing by half 
2. Normalisation

In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from skimage import io, transform
import torch.optim as optim
import os
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import cv2

torch.backends.cudnn.deterministic=True
torch.cuda.manual_seed_all(999)
torch.manual_seed(999)
np.random.seed(999)



In [3]:
MODEL="model69000_batch_64_.pt"
RESULTS_FILE="results_model69000_batch_64__mach_1.csv"
BATCH_SIZE=16




class FlipkartDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.coords_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.coords_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.coords_frame.loc[idx,:]["image_name"])
        image = cv2.imread(img_name)
        image = cv2.resize(image, dsize=(320,240), interpolation=cv2.INTER_CUBIC)
        image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        coords=[]
        
        coords=torch.tensor(np.array([self.coords_frame.loc[idx,:]["x1"],
                self.coords_frame.loc[idx,:]["y1"],
                self.coords_frame.loc[idx,:]["x2"],
                self.coords_frame.loc[idx,:]["y2"]]))
        
        img_name= self.coords_frame.loc[idx,:]["image_name"]
        X=dict({"image":image,"name":img_name})
        y=coords
        
        return X,y
    

params = {'batch_size': BATCH_SIZE,
          'shuffle': False}

sampler=FlipkartDataset("Dataset/test.csv","Dataset/images/")
testing_generator = data.DataLoader(sampler, **params)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # input dim (3*240*320)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
        self.conv1_bn = nn.BatchNorm2d(num_features=16)
        # 16*240*320
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2,padding=1)
        # 16*120*160
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3,)
        self.conv2_bn = nn.BatchNorm2d(num_features=32)
        # 32*120*160
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2,padding=1)
        # 32*60*80
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3_bn = nn.BatchNorm2d(num_features=64)
        # 64*60*80
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2,padding=1)
        # 64*30*40
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv4_bn = nn.BatchNorm2d(num_features=128)
        # 128*30*40
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2,padding=1)
        # 128*15*20
        
        
        self.fc1 = nn.Linear(128*15*20,2000)
        self.fc2 = nn.Linear(2000, 1000)
        self.fc3 = nn.Linear(1000, 500)
        self.fc4 = nn.Linear(500, 4)
        

    def forward(self, x):
        x = self.pool1(self.conv1_bn(F.relu(self.conv1(x))))
        x = self.pool2(self.conv2_bn(F.relu(self.conv2(x))))
        x = self.pool3(self.conv3_bn(F.relu(self.conv3(x))))
        x = self.pool4(self.conv4_bn(F.relu(self.conv4(x))))
        #x = self.pool5(F.relu(self.conv5(x)))
        #x = self.pool5(F.relu(self.conv5_bn(self.conv5(x))))
        x = x.view(-1,128*15*20)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [5]:
net=Net().cuda()

checkpoint = torch.load("models/Mach_1/"+MODEL)
net.load_state_dict(checkpoint['state_dict'])
net.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv1_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=38

In [6]:

img_idx=list()
x1_idx=list()
y1_idx=list()
x2_idx=list()
y2_idx=list()


z=0

for X,y in testing_generator:
    for each in X["name"]:
        img_idx.append(str(each))
    X = X["image"].type('torch.FloatTensor') 
    X=X.transpose(3,1)
    X=X.transpose(2,3)
    X=X.cuda()
    
    
    output=net(X)
    
    for each in output:
        x1_idx.append(int(each[0]))
        y1_idx.append(int(each[1]))
        x2_idx.append(int(each[2]))
        y2_idx.append(int(each[3]))
    
    clear_output()
    z+=BATCH_SIZE
    print("Completed:",z,"/24046")
    
print("Writing to ",RESULTS_FILE)
grid=pd.DataFrame({"image_name":img_idx,"x1":x1_idx,"x2":x2_idx,"y1":y1_idx,"y2":y2_idx})
grid.to_csv(RESULTS_FILE,index=None)


Completed: 24048 /24046
Writing to  results_model69000_batch_64__mach_1.csv
